In [10]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.typing import NDArray
from scipy.special import comb
from importlib import reload

In [11]:
# physical parameters: optical linewidth (gamma), spin coherence time of emitter (t_coh), ancilla coherence time
# Global variables in uppercase
GAMMA = 2e9*2*np.pi # Hz
T_SPIN_COHERENCE = 13e-3 # s
T_ANCILLA_COHERENCE = np.inf
DEPTH = 3

class Miu:
    def __init__(self, L, L_att, L_delay, m, n_f=np.nan):
        self._L = L
        self._L_att = L_att
        self.L_delay = L_delay  # does this depends on the position of qubit? No, it is assuming the worst case
        self.m = m
        self.n_f = n_f
        self.coup = 0.05
        self.int_ancilla = 0
        self.depth = DEPTH

    @property
    def ext_tree(self):
        return 1 - np.exp(-self._L/(self.m+1)/self._L_att)
    @property
    def ext_RGS(self):
        return 1 - np.exp(-self._L/2/(self.m+1)/self._L_att)
    
    def int_feedback(self, L_f): # not sure how to implement this (number of roundtrip?)
        return 1 - np.exp(-self.n_f*L_f/self._L_att)
    
    @property
    def delay(self):
        return  1 - np.exp(-self.L_delay/self._L_att)

    @property
    def total_tree_a(self):
        return 1 - (1-self.ext_tree)*(1-self.coup)*(1-self.int_ancilla)*(1-self.delay)

    def total_tree_f(self):
        ...


EPSILON_DEPOLARIZATION = 5e-5
BETA = 500.

# time for operations:
class Time:
    def __init__(self, gamma, beta):
        self._gamma = gamma
        self._beta = beta 

        self.H = 100e-12  # s
        self.CZ_a = 100e-9 # s
        self.CZ_f = self.E_f # s
    
    @property
    def gamma(self):
        return self._gamma
    @property
    def P_a(self):
        return 1/self._gamma
    @property
    def P_f(self):
        return 1/self._gamma
    @property
    def M(self):
        return 10*self.P_a
    @property
    def E_a(self):    
        return self.P_a + self.H + self.M
    @property
    def E_f(self):
        return self._beta*self.P_f + self.H + self.M
# setup parameters: distance between A&B (L), attenuation distance (L_att), speed of light in delay line (v_delay)
L = 1000e3 # m
L_ATT = 20e3 # m
V_DELAY = 3e8 # m/s # seems inconsistent??

# variable paramaters: m, branching parameter {b_i}

In [12]:
m = Miu(1000e3, 20e3, 100, 499)
print(m.total_tree_a)

0.14469170354304772


In [13]:
# tree properties
class Tree:
    def __init__(self, branch_param: NDArray, miu: float=0) -> None:
        self.b = branch_param # array 
        self.depth = len(self.b)
        self.miu = miu # same for all photons? Assume yes

        self.n_total = self.num_qubits(1, self.depth)

        # Probability of obtaining an outcome from an indirect Z measurement of any photon in i-th level of the tree (eq. 5)
        # Here is 1 indexed (same as paper)
        self.R = np.zeros(self.depth)
        self.R[0] = np.nan

        for i in range(self.depth-1, 0, -1): # d-1 ... 1

            if i == self.depth-1: 
                R_i_plus_2 = 0
                b_i_plus_1 = 0 # no qubits at depth+1 level
            elif i == self.depth-2: 
                R_i_plus_2 = 0 # qubits at the last level cannot be indirectly measured
                b_i_plus_1 = self.b[i+1] 
            else:
                b_i_plus_1 = self.b[i+1] 
                R_i_plus_2 = self.R[i+2] 
            # print(R_i_plus_2)
            # print(b_i_plus_1)
            self.R[i] = 1 - (1 - (1-miu)*(1-miu+miu*R_i_plus_2)**b_i_plus_1)**self.b[i] # confirm this

        # Success probability between neighbouring nodes (eq. 3, without ^m+1)
        if self.depth >= 3:
            R_2 = self.R[2] 
        else:   
            R_2 = 0
        self.P_succ = ((1-miu+miu*self.R[1])**self.b[0] - (miu*self.R[1])**self.b[0])*(1-miu+miu*R_2)**self.b[1]
    
    

    # Number of qubits from depth i to j 
    def num_qubits(self, start: int, end: int) -> int:
        return np.sum([np.product(self.b[0:i]) for i in range(start, end+1)])
    
    
# Generation time (s)
def T_tree_a(tree: Tree, time: Time):
    num_last_layer = tree.num_qubits(tree.depth,tree.depth)
    num_2nd_to_d_minus_1 = tree.num_qubits(2,tree.depth-1)
    num_1st_to_d_minus_1 = tree.num_qubits(1,tree.depth-1)
    return num_last_layer*time.P_a + (time._beta*tree.b[0] + num_2nd_to_d_minus_1)*time.E_a + num_1st_to_d_minus_1*time.CZ_a



In [15]:
def Ta2(tree: Tree, time: Time):

    return tree.b[1]*tree.b[2]*time.P_a + (time._beta + tree.b[1])*time.E_a + tree.b[0]*tree.b[1]*time.CZ_a # time part in eq. 20 in appendix D, depth = 3

# Based on result in p.18
v_delay = 2e8
# GAMMA = 2e9
GAMMA = 2e9*2*np.pi
tree = Tree([4,16,5])
time = Time(GAMMA, beta=500)
print("time for gates:", time.P_a, time.E_a, time.CZ_a)
# print("v_delay:", 398/Ta2(tree, time)/3e8) #L_delay/time_part
# print("v_delay:", 398/Ta2(tree, time)/3e8) #L_delay/time_part
print("v_delay:", Ta2(tree, time)*v_delay) #L_delay/time_part

# GAMMA = 100e9
GAMMA = 100e9*2*np.pi
tree = Tree([1,1,19])
time = Time(GAMMA, beta=500)
print("time for gates:", time.P_a, time.E_a, time.CZ_a)
# print("v_delay:", 80.4/Ta2(tree, time)/3e8)
print("v_delay:", Ta2(tree, time)*v_delay) #L_delay/time_part


GAMMA = 170e6
# GAMMA = 170e6*2*np.pi
tree = Tree([4,18,15])
time = Time(GAMMA, beta=500)
print("time for gates:", time.P_a, time.E_a, time.CZ_a)
# print("v_delay:", 627.9/Ta2(tree, time)/3e8)
print("v_delay:", Ta2(tree, time)*v_delay) #L_delay/time_part
# GAMMA = 100e9
GAMMA = 100e9*2*np.pi
tree = Tree([4,16,15])
time = Time(GAMMA, beta=500)
print("time for gates:", time.P_a, time.E_a, time.CZ_a)
# print("v_delay:", 380.7/Ta2(tree, time)/3e8)
print("v_delay:", Ta2(tree, time)*v_delay) #L_delay/time_part

time for gates: 7.957747154594768e-11 9.753521870054244e-10 1e-07
v_delay: 1381.9295852436949
time for gates: 1.5915494309189534e-12 1.175070437401085e-10 1e-07
v_delay: 31.780253670596363
time for gates: 5.88235294117647e-09 6.480588235294117e-08 1e-07
v_delay: 8471.536470588235
time for gates: 1.5915494309189534e-12 1.175070437401085e-10 1e-07
v_delay: 1292.2031212866632
